In [641]:
import re
from copy import deepcopy

file_path = 'YFJS04'
with open(file_path, 'r') as file:
    lines = file.readlines()

In [642]:
operations_machines_times = dict()
successor_operations = dict()
operation_machine_times = dict()
for i in range(0, len(lines)):
    line = lines[i].strip()
    if i == 0:
        match = re.search(r'\d+$', line)
        n_jobs = int(match.group())
    elif i == 1:
        match = re.search(r'\d+$', line)
        n_operations_per_job = int(match.group())
    elif i == 2:
        match = re.search(r'\d+$', line)
        n_machines = int(match.group())
    elif i == 3:
        match = re.search(r'\d+$', line)
        nmax_machines_per_operation = int(match.group())
    elif i == 4:
        matches = re.findall(r'\d+', line)
        matches = [int(x) for x in matches]
        n_operations = int(matches[0])
        n_conditions = int(matches[1])
    elif i > 4 and i < n_conditions + 5:
        matches = re.findall(r'\d+', line)
        matches = [int(x) for x in matches]
        if successor_operations.get(matches[0], False):
            successor_operations[matches[0]].append(matches[1])
        else:
            successor_operations[matches[0]] = [matches[1]]
        _ = i
    else:
        matches = re.findall(r'\d+', line)
        matches = [int(x) for x in matches]
        operation = i - _ - 1
        temp = {}
        for j in range(1, len(matches), 2):
            temp[matches[j]] = matches[j+1]
        operation_machine_times[operation] = temp


predecessor_operations = dict()
for key, values in successor_operations.items():
    temp = []
    for value in values:
        if value in predecessor_operations:
            predecessor_operations[value].append(key)
        else:
            predecessor_operations[value] = [key] 

In [643]:
rank = dict()

def compute_rank(operation):
    machine_times = operation_machine_times[operation].values()
    average_machine_time = sum(machine_times)/len(machine_times)
    rank[operation] = average_machine_time
    for successor in successor_operations.get(operation, []):
        rank[operation] = max(rank[operation], average_machine_time + compute_rank(successor))
    return rank[operation]

for operation in range(n_operations):
    if operation not in rank:
        compute_rank(operation)

In [644]:
class DAG:
    id = 0
    def __init__(self, operations) -> None:
        self.operations = operations
        for operation in self.operations:
            operation.setDag(self)
        self.D_OPQ = self.operations
        self.dagId = DAG.id
        DAG.id += 1
    def addNodes(self, nodes):
        pass

class Operation:
    def __init__(self, operationId, machineTimes):
        self.operationId = operationId
        self.status = "incomplete"
        self.machineTimes = machineTimes
        self.timeLeft = None
    
    def setDag(self, dag):
        self.dag = dag
    
    def getParent(self):
        return self.dag

    def getStatus(self):
        return self.status

    def setStatus(self, status):
        self.status = status

    def getOperationId(self):
        return self.operationId

    def getMachineTimes(self):
        return self.machineTimes
    
    def getMachineIdByLowestTime(self):
        machineId = None
        lowestTime = float('inf')
        for machine, time in self.machineTimes.items():
            if time < lowestTime:
                machineId = machine
                lowestTime = time
        self.timeLeft = lowestTime
        return machineId

dags = []
operations = []
for job in range(n_jobs):
    temp = []
    for i in range(n_operations_per_job):
        temp.append(Operation(n_operations_per_job*job+i, operation_machine_times[n_operations_per_job*job+i]))
    operations.extend(temp)
    dags.append(DAG(sorted(temp, key = lambda x: rank[x.operationId], reverse=False)))        

In [645]:
class Machine:
    def __init__(self, machineId):
        self.M_OCQ = []
        self.machineId = machineId
        self.status = "idle"
        self.scheduledOperation = None
    
    def scheduleOperationtoQueue(self, operation):
        self.M_OCQ.insert(0,operation)
    
    def setStatus(self, status):
        self.status = status
    
    def getStatus(self):
        return self.status
    
    def getScheduledOperation(self):
        return self.scheduledOperation
    
    def getPredecessors(self, operationId):
        predecessors = []
        for predecessorId in predecessor_operations.get(operationId, []):
            predecessors.append(operations[predecessorId])
        return predecessors

    def scheduleOperationToMachine(self):
        if self.M_OCQ: 
            if self.getStatus() == "idle":
                self.scheduledOperation = self.M_OCQ[-1]
                if self.checkPredecessorOperationsStatus(self.M_OCQ[-1]) == "complete":
                    self.setStatus("inProgress")
                else:
                    self.setStatus("waiting")
            elif self.getStatus() == "inProgress":
                self.updateTimeLeft()
            elif self.getStatus() == "waiting":
                if self.checkPredecessorOperationsStatus(self.M_OCQ[-1]) == "complete": #To be implemented. Check if predecessor is done. If yes, then update the status
                    self.setStatus("inProgress")
            return True
        else:
            return False
        
    def updateTimeLeft(self):
        self.scheduledOperation.timeLeft -= 1
        if self.scheduledOperation.timeLeft == 0:
            self.setStatus("idle")
            self.scheduledOperation.setStatus("complete")
            self.M_OCQ.pop()
            
    
    def checkPredecessorOperationsStatus(self, operation):
        predecessors = self.getPredecessors(operation.getOperationId())
        for predecessor in predecessors:
            if predecessor.getStatus() == "incomplete":
                return "incomplete"
        return "complete"

machines = []
for i in range(n_machines):
    machines.append(Machine(i))

CBQ = []
operationsLeft = True
while operationsLeft:
    CBQ.clear()
    for dag in dags:
        operationsLeft = False
        if dag.D_OPQ:
            operation = dag.D_OPQ.pop()
            CBQ.append(operation)
            operationsLeft = True
    CBQ = sorted(CBQ, key = lambda x: rank[x.operationId], reverse = False)
    while CBQ:
        operation = CBQ.pop() #select operation with maximum rank
        machines[operation.getMachineIdByLowestTime()].scheduleOperationtoQueue(operation) #assign it to machine with lowest time to process it

In [646]:
time = 0
for machine in machines:
    machine.scheduleOperationToMachine()

operationsLeft = True
while operationsLeft:
    operationsLeft = False
    for machine in machines:
        x = machine.scheduleOperationToMachine()
        operationsLeft = x or operationsLeft
    time = time + 1

print(time)

624
